# Lab02

In [154]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math

In [155]:
def fst_vec_norm(x: np.ndarray):
    return max(abs(x))

def scd_vec_norm(x: np.ndarray):
    return sum(abs(x))

def trd_vec_norm(x: np.ndarray):
    return math.sqrt(np.dot(x, x))

In [156]:
def fst_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A[i])) for i in range(A.shape[0])])

def scd_m_norm(A: np.ndarray):
    assert(A.shape[0] == A.shape[1])
    return max([sum(abs(A.T[i])) for i in range(A.T.shape[0])])

#! уточнить, как в данной работе мы ищем собственные числа
def trd_m_norm(A: np.ndarray):
    pass

In [157]:
# порядок округдения коэффициентов (нужен исключительно для вывода)
# при вычислениях коэффициенты НЕ ОКРУГЛЯЮТСЯ
round_n = 3

class Slae:
    def __init__(self, matrix: np.ndarray, values: np.ndarray):

        # проверяем, что матрица квадратная и вектор значений имеет соответсвующую размерность
        assert(matrix.shape[0] == matrix.shape[1])
        assert(matrix.shape[0] == values.shape[0])
        
        self.A = matrix
        self.f = values

    @property
    def dimention(self):
        return self.A.shape[0]

    def check_symmetric(self, a, tol=1e-16):
        return not False in (np.abs(self.A-self.A.T) < tol)

    def Gauss_mthd(self):
        for k in range(self.dimention):
            for m in range(k+1, self.dimention):

                alpha = self.A[m][k] / self.A[k][k]

                self.f[m] = self.f[m] - self.f[k] * alpha 
                for i in range(k, self.dimention):
                    self.A[m][i] = self.A[m][i] - self.A[k][i] * alpha


        solution = np.full((self.dimention, ), 0.0)
        solution[self.dimention-1] = self.f[self.dimention-1] / self.A[self.dimention-1][self.dimention-1]

        for k in range(self.dimention-2, 0-1, -1):
            solution[k] = 1 / self.A[k][k] * (self.f[k] - np.dot(self.A[k], solution))

        return solution

    def LU_mthd(self):
        pass

    def Holecky_mthd(self):
        pass

    def Zaydel_mthd(self):
        pass

    ## overloading output
    def __str__(self):
        res = ''
        for i in range(self.A.shape[0]):
            string = ''
            for j in range(self.A.shape[1]):
                string = string + str(round(self.A[i][j], round_n)) + ' x{}'.format(j + 1)
                # string = string + str(self.A[i][j]) + ' x{}'.format(j + 1)
                if j != self.A.shape[1] - 1:
                    string = string + ' + '
                else:
                    string = string + ' = ' + str(round(self.f[i], round_n))
                    # string = string + ' = ' + str(self.f[i])
            string = string + '\n'
            res = res + string

        return res

In [158]:
N = 12
A = np.eye(N)
f = np.full((N, ), 1.0)


for i in range(N):
    for j in range(N):
        if i == j:
            A[i][j] = 1
        else:
            A[i][j] = 1 / ((i+1)**2 + (j+2))
    f[i] = 1 / (i + 1)

# eq = Slae(A, f)

In [160]:
a = np.array([
    [1, 2, 1],
    [2, 1, 2],
    [3, 3, 1]
])

f = np.array([8, 10, 12])

eq = Slae(a, f)
print(eq)


sol = eq.Gauss_mthd()
print(eq)
print('sol = ', sol)

1 x1 + 2 x2 + 1 x3 = 8
2 x1 + 1 x2 + 2 x3 = 10
3 x1 + 3 x2 + 1 x3 = 12

1 x1 + 2 x2 + 1 x3 = 8
0 x1 + -3 x2 + 0 x3 = -6
0 x1 + 0 x2 + -2 x3 = -6

sol =  [1. 2. 3.]
